In [1]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt

plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [2]:
from pathlib import Path
path = Path.cwd()
data_path = path.parent
data_path = str(data_path) + '/data/'
data_path

'/Users/keumhyeonjun/Projects/행안부/알고리즘/data/'

In [3]:
# whether csv read, 일시를 index로 넣고 datetime 형태로 변환함
whether = pd.read_csv(data_path +'Whether.csv', encoding='euc-kr', parse_dates=['일시'], index_col='일시')
# 강수량, 적설이 null인 경우 모두 0으로 바꿈
whether = whether.fillna(0)
# 일시의 freq을 hour로 변경
whether.index.freq = 'h'

'''
데이터가 누락된 시간을 확인하기 위한 부분 (파일로 받은 경우)
API로 받는 경우 날짜는 있지만 모든 데이터가 Null값이기 때문에 해당 조건으로 수정해야 할 것으로 보임

j = whether.index[0]
for i in whether.index:
    if i != j + pd.Timedelta("1 hour"):
        print(j, i)
    j = i
'''
# 지점코드, 지점명을 없앰
whether = whether.iloc[:, [2,3,4,5,6]]
whether

,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm)
일시,,,,,
2020-10-01 00:00:00,17.3,0.0,0.2,92.0,0.0
2020-10-01 01:00:00,16.8,0.0,0.5,94.0,0.0
2020-10-01 02:00:00,16.9,0.0,1.1,93.0,0.0
2020-10-01 03:00:00,16.8,0.0,0.4,94.0,0.0
2020-10-01 04:00:00,16.1,0.0,0.1,95.0,0.0
...,...,...,...,...,...
2022-10-01 19:00:00,21.1,0.0,0.1,54.0,0.0
2022-10-01 20:00:00,19.6,0.0,0.6,63.0,0.0
2022-10-01 21:00:00,18.8,0.0,0.7,73.0,0.0


# 차량 데이터 불러오기 및 전처리

# 다래데이터만 > 다래 데이터 형식으로 테이블 맞춰서 제공한다고 하였음

In [4]:
import pandas as pd

In [5]:
darae_in = pd.read_csv(data_path+ 'VW_InCarinfo (차량 입차 정보 조회).csv')
darae_in = darae_in.dropna()
darae_out = pd.read_csv(data_path+'VW_OutCarinfo (차량 출차 정보 조회).csv')

/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_67271/1372231495.py:1: DtypeWarning: Columns (0,1,2,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  darae_in = pd.read_csv(data_path+ 'VW_InCarinfo (차량 입차 정보 조회).csv')


In [6]:
darae_in[['입차고객구분']]= darae_in[['입차고객구분']].apply(lambda x: x.str.strip(), axis = 1)
darae_in[['장비명','입차구분','입차고객구분','출차여부','차량종류']] = darae_in[['장비명','입차구분','입차고객구분','출차여부','차량종류']].apply(lambda x: x.str.strip(), axis = 1)
darae_out[['장비명','출차구분','출차고객구분','출차여부','차량종류']]= darae_out[['장비명','출차구분','출차고객구분','출차여부','차량종류']].apply(lambda x: x.str.strip(), axis = 1)

In [7]:
darae_in['장비명'].value_counts()

12동입구        7162
14-2동입구      7161
14-2동지하입구    4779
15동지하입구      4394
13-1동입구      4005
12동지하입구      2586
13-2동입구      2558
13-2동지하입구    1926
15동입구        1763
14-1동입구       879
13-3동입구       839
13-1동지하입구     620
Name: 장비명, dtype: int64

In [8]:
darae_out

,출차일자,출차시각,차량번호,주차일련번호,출차카드번호,장비명,출차구분,출차고객구분,출차여부,차량종류
0,2022-09-01,05:32:24,94고6836,1.111422e+16,1.000000e+14,13-3동출구#1,정기권출차,정기권,출차,소형
1,2022-09-01,05:52:37,94고6836,1.111422e+16,1.000000e+14,13-3동출구#1,정기권출차,정기권,출차,소형
2,2022-09-01,06:12:11,94고6836,1.111422e+16,1.000000e+14,13-3동출구#1,정기권출차,정기권,출차,소형
3,2022-09-01,06:18:27,82도4072,1.111422e+16,1.000000e+14,13-3동출구#1,정기권출차,정기권,출차,소형
4,2022-09-01,06:39:34,94고6836,1.111422e+16,1.000000e+14,13-3동출구#1,정기권출차,정기권,출차,소형
...,...,...,...,...,...,...,...,...,...,...
38667,2022-10-05,17:40:15,71하3305,1.411220e+16,1.000000e+14,15동지하입구,정상,정기권,미출차,소형
38668,2022-10-05,19:03:47,50머4665,1.411220e+16,1.000000e+14,15동지하입구,정상,정기권,미출차,소형
38669,2022-10-05,19:44:02,323러9283,1.411220e+16,1.000000e+14,15동지하입구,정상,정기권,미출차,소형
38670,2022-10-05,19:46:40,09도7906,1.411220e+16,1.000000e+14,15동지하입구,정상,정기권,미출차,소형


## 이름 바꾸기 13_1동 지하 1 > 13_1동

In [9]:
import re
def slicing(df,i) :
    pattern = '.*동'
    reg = re.search(pattern, df['장비명'][i])
    reg = reg.group()
    df['주차장'][i] = reg
    return reg

darae_in['주차장'] = 0
for i in range(len(darae_in)):
    slicing(darae_in,i)
darae_out['주차장'] = 0
for i in range(len(darae_out)):
    slicing(darae_out,i)

def change(x):
    if x =='13-1동' or x =='13-2동' or x =='13-3동':
        return '13동'
    elif x == '14-1동' or x == '14-2동':
        return '14동'
    else :
        return x
darae_in['주차장'] = darae_in['주차장'].apply(change)
darae_out['주차장'] = darae_out['주차장'].apply(change)

/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_67271/3206655803.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['주차장'][i] = reg
/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_67271/3206655803.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['주차장'][i] = reg


In [10]:
pd.options.display.float_format = '{:.0f}'.format

In [11]:
import copy

In [12]:
test_in = copy.deepcopy(darae_in)
test_in.columns = ['일자','시각','차량번호','주차일련번호','입출차카드번호','장비명','입출차구분','입출차고객구분','출차여부','차량종류','주차장']
test_in['입출차여부'] = '입차'

In [13]:
test_out = copy.deepcopy(darae_out)
test_out.columns = ['일자','시각','차량번호','주차일련번호','입출차카드번호','장비명','입출차구분','입출차고객구분','출차여부','차량종류','주차장']
test_out['입출차여부'] = '출차'
test_out[test_out['장비명'].str.contains('입구')]['입출차카드번호'].value_counts()

100000000000000    15818
0                     35
Name: 입출차카드번호, dtype: int64

In [14]:
df = pd.concat([test_in, test_out]).sort_values(by=['차량번호','일자','시각'])#[['차량번호','입출차여부','일자','시각','입출차구분','출차여부','장비명']]

In [15]:
import datetime
df['입출차시각'] = df['일자'] + df['시각']
df['입출차시각'] = pd.to_datetime(df['입출차시각'], format= '%Y-%m-%d %H:%M:%S', errors='raise')
df['시'] = df['입출차시각'].dt.hour
df.reset_index(inplace = True)

In [16]:
df

,index,일자,시각,차량번호,주차일련번호,입출차카드번호,장비명,입출차구분,입출차고객구분,출차여부,차량종류,주차장,입출차여부,입출차시각,시
0,4218,2022-09-05,09:07:56,01거5868,11111220905090756,100000000037794,12동입구,정상,정기권,출차,소형,12동,입차,2022-09-05 09:07:56,9
1,2114,2022-09-05,09:32:50,01거5868,11111220905090750,100000000037794,12동출구#1,정기권출차,정기권,출차,소형,12동,출차,2022-09-05 09:32:50,9
2,4350,2022-09-05,17:00:27,01거5868,11111220905170028,100000000037794,12동입구,정상,정기권,출차,소형,12동,입차,2022-09-05 17:00:27,17
3,2588,2022-09-05,17:27:22,01거5868,11111220905170020,100000000037794,12동출구#1,정기권출차,정기권,출차,소형,12동,출차,2022-09-05 17:27:22,17
4,4554,2022-09-06,09:02:53,01거5868,11111220906090252,100000000037794,12동입구,정상,정기권,출차,소형,12동,입차,2022-09-06 09:02:53,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77339,22206,2022-10-05,15:29:16,충북80배1724,11112221005150810,100000000039079,13-1동출구#1,정기권출차,정기권,출차,소형,13동,출차,2022-10-05 15:29:16,15
77340,9973,2022-09-10,16:07:19,충북86자3167,11112220910160720,100000000051701,13-1동입구,정상,정기권,강제출차,소형,13동,입차,2022-09-10 16:07:19,16
77341,30326,2022-09-28,17:26:49,충북86자3167,11113220928172648,100000000051701,13-2동입구,중복,정기권,출차,소형,13동,입차,2022-09-28 17:26:49,17
77342,30326,2022-09-28,17:26:49,충북86자3167,11113200000000000,100000000000000,13-2동입구,중복,정기권,출차,소형,13동,출차,2022-09-28 17:26:49,17


# 입차시간 == 출차시간인 경우 파악하기
- 입차시간 == 출차시간인 경우 입차로간주하고 출차 제거하기

In [17]:
car_arr = np.array(df[['일자','시각','차량번호']])
car_arr

array([['2022-09-05', ' 09:07:56', '01거5868'],
       ['2022-09-05', ' 09:32:50 ', '01거5868'],
       ['2022-09-05', ' 17:00:27', '01거5868'],
       ...,
       ['2022-09-28', ' 17:26:49', '충북86자3167'],
       ['2022-09-28', ' 17:26:49', '충북86자3167'],
       ['2022-09-28', ' 17:54:17 ', '충북86자3167']], dtype=object)

In [18]:
test_idx_list =[]
for idx in range(len(car_arr)-1):
    if car_arr[idx][0] == car_arr[idx+1][0] and car_arr[idx][1] == car_arr[idx+1][1] and car_arr[idx][2] == car_arr[idx+1][2]:
        test_idx_list.append(idx)
        test_idx_list.append(idx+1)
        #test_idx_list.append(idx+2)
test_idx_list = list(set(test_idx_list))
test_df = df.loc[test_idx_list]
eleminate_index = test_df[test_df['입출차여부']=='출차'].index
df2 = df.drop(eleminate_index)
df2.drop(['index'],axis = 1 , inplace = True)
df2.reset_index(inplace=True)
df2.drop(['index'],axis = 1 , inplace = True)
df2

,일자,시각,차량번호,주차일련번호,입출차카드번호,장비명,입출차구분,입출차고객구분,출차여부,차량종류,주차장,입출차여부,입출차시각,시
0,2022-09-05,09:07:56,01거5868,11111220905090756,100000000037794,12동입구,정상,정기권,출차,소형,12동,입차,2022-09-05 09:07:56,9
1,2022-09-05,09:32:50,01거5868,11111220905090750,100000000037794,12동출구#1,정기권출차,정기권,출차,소형,12동,출차,2022-09-05 09:32:50,9
2,2022-09-05,17:00:27,01거5868,11111220905170028,100000000037794,12동입구,정상,정기권,출차,소형,12동,입차,2022-09-05 17:00:27,17
3,2022-09-05,17:27:22,01거5868,11111220905170020,100000000037794,12동출구#1,정기권출차,정기권,출차,소형,12동,출차,2022-09-05 17:27:22,17
4,2022-09-06,09:02:53,01거5868,11111220906090252,100000000037794,12동입구,정상,정기권,출차,소형,12동,입차,2022-09-06 09:02:53,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61486,2022-10-05,15:08:15,충북80배1724,11112221005150816,100000000039079,13-1동입구,정상,정기권,출차,소형,13동,입차,2022-10-05 15:08:15,15
61487,2022-10-05,15:29:16,충북80배1724,11112221005150810,100000000039079,13-1동출구#1,정기권출차,정기권,출차,소형,13동,출차,2022-10-05 15:29:16,15
61488,2022-09-10,16:07:19,충북86자3167,11112220910160720,100000000051701,13-1동입구,정상,정기권,강제출차,소형,13동,입차,2022-09-10 16:07:19,16
61489,2022-09-28,17:26:49,충북86자3167,11113220928172648,100000000051701,13-2동입구,중복,정기권,출차,소형,13동,입차,2022-09-28 17:26:49,17


In [19]:
test_df[test_df['차량번호'] == '85부3062']


,index,일자,시각,차량번호,주차일련번호,입출차카드번호,장비명,입출차구분,입출차고객구분,출차여부,차량종류,주차장,입출차여부,입출차시각,시
73652,24350,2022-09-16,07:59:10,85부3062,14111220916075910,100000000024581,14-2동지하입구,중복,정기권,출차,소형,14동,입차,2022-09-16 07:59:10,7
73653,24350,2022-09-16,07:59:10,85부3062,14111200000000000,100000000000000,14-2동지하입구,중복,정기권,출차,소형,14동,출차,2022-09-16 07:59:10,7
73661,24494,2022-09-16,13:55:30,85부3062,14111220916135530,100000000024581,14-2동지하입구,중복,정기권,출차,소형,14동,입차,2022-09-16 13:55:30,13
73662,24494,2022-09-16,13:55:30,85부3062,14111200000000000,100000000000000,14-2동지하입구,중복,정기권,출차,소형,14동,출차,2022-09-16 13:55:30,13
73672,24593,2022-09-19,08:00:10,85부3062,14111220919080010,100000000024581,14-2동지하입구,정상,정기권,출차,소형,14동,입차,2022-09-19 08:00:10,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73945,30898,2022-10-05,08:03:15,85부3062,11114200000000000,100000000000000,13-3동입구,정상,정기권,출차,소형,13동,출차,2022-10-05 08:03:15,8
73947,37314,2022-10-05,08:12:25,85부3062,11112221005081224,100000000024581,13-1동입구,정상,정기권,출차,소형,13동,입차,2022-10-05 08:12:25,8
73948,37314,2022-10-05,08:12:25,85부3062,11112200000000000,100000000000000,13-1동입구,정상,정기권,출차,소형,13동,출차,2022-10-05 08:12:25,8
73951,37383,2022-10-05,15:33:40,85부3062,11112221005153340,100000000024581,13-1동입구,정상,정기권,출차,소형,13동,입차,2022-10-05 15:33:40,15


# 데이터 전처리
- 아침시간이니 입차시간 == 출차시간인 경우 입차로 가정
- 차량 번호별로 입차1,입차2,.....출차,출차 인경우  >>  입차1, 출차로
- 입차1,입차2,입차3, 출차 >>> 입차3, 출차 로해야하는게 맞는듯?
- 아니면 출차 주차장 기준으로 하든가

## 입차입차 연속인 경우 > 입차 출차 입차로 바꾸기????

In [23]:
car_arr = np.array(df2[['차량번호','입출차여부','시각']])
test_idx_list =[]
for idx in range(len(car_arr)-1):
    if car_arr[idx][0] == car_arr[idx+1][0] and car_arr[idx][1] == '입차' and car_arr[idx+1][1]=='입차':
        test_idx_list.append(idx)
        test_idx_list.append(idx+1)
test_idx_list = list(set(test_idx_list))
df2.loc[test_idx_list]

,일자,시각,차량번호,주차일련번호,입출차카드번호,장비명,입출차구분,입출차고객구분,출차여부,차량종류,주차장,입출차여부,입출차시각,시
10,2022-09-01,07:42:33,01나3085,11113220901074232,100000000029503,13-2동입구,정상,정기권,강제출차,소형,13동,입차,2022-09-01 07:42:33,7
11,2022-09-08,08:05:47,01나3085,11113220908080548,100000000029503,13-2동입구,중복,정기권,강제출차,소형,13동,입차,2022-09-08 08:05:47,8
12,2022-09-08,10:02:42,01나3085,11113220908100242,100000000029503,13-2동입구,중복,정기권,강제출차,소형,13동,입차,2022-09-08 10:02:42,10
13,2022-09-13,16:49:44,01나3085,11112220913164944,100000000029503,13-1동입구,중복,정기권,출차,소형,13동,입차,2022-09-13 16:49:44,16
23,2022-09-26,17:52:40,01나3085,11112220926175240,100000000029503,13-1동입구,정상,정기권,강제출차,소형,13동,입차,2022-09-26 17:52:40,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61480,2022-10-05,14:34:43,충북80배1724,11111221005143444,100000000039079,12동입구,중복,정기권,출차,소형,12동,입차,2022-10-05 14:34:43,14
61481,2022-10-05,14:37:02,충북80배1724,13111221005143702,100000000039079,14-1동입구,정상,정기권,출차,소형,14동,입차,2022-10-05 14:37:02,14
61482,2022-10-05,14:43:12,충북80배1724,13112221005144312,100000000039079,14-2동입구,정상,정기권,출차,소형,14동,입차,2022-10-05 14:43:12,14
61488,2022-09-10,16:07:19,충북86자3167,11112220910160720,100000000051701,13-1동입구,정상,정기권,강제출차,소형,13동,입차,2022-09-10 16:07:19,16
